In [1]:
import json
import random

from pprint import pprint

from datasets import Dataset, load_dataset
from tqdm.notebook import tqdm

In [2]:
SYSTEM_PROMPT = \
"""
Ты — ассистент, который помогает человеку делать суммаризацию текста.\
Твоя задача — формулировать краткое изложение на русском языке, \
точно передавая смысл без искажений и отклонений от оригинала.
""".strip()

In [3]:
def dataset_to_jsonl(
    dataset: Dataset,
    system_prompt: str,
    prompt_column: str,
    response_column: str,
) -> list[dict]:
    lines = []
    for sample in tqdm(dataset):
        item = {
            "messages": [
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": sample[prompt_column].strip()},
                {"role": "assistant", "content": sample[response_column].strip()}
            ]
        }
        lines.append(item)
    
    return lines

In [4]:
train_dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset", split="train")
test_dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset", split="test")

train/train.parquet:   0%|          | 0.00/538M [00:00<?, ?B/s]

test/test.parquet:   0%|          | 0.00/605k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/197561 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/258 [00:00<?, ? examples/s]

In [5]:
print(train_dataset)

Dataset({
    features: ['text', 'summary'],
    num_rows: 197561
})


In [6]:
prompt_column="text"
response_column="summary"

train_jsonl = dataset_to_jsonl(
    train_dataset,
    system_prompt=SYSTEM_PROMPT,
    prompt_column=prompt_column,
    response_column=response_column
)

test_jsonl = dataset_to_jsonl(
    test_dataset,
    system_prompt=SYSTEM_PROMPT,
    prompt_column=prompt_column,
    response_column=response_column
)

  0%|          | 0/197561 [00:00<?, ?it/s]

  0%|          | 0/258 [00:00<?, ?it/s]

In [7]:
pprint(train_jsonl[0])

{'messages': [{'content': 'Ты — ассистент, который помогает человеку делать '
                          'суммаризацию текста.Твоя задача — формулировать '
                          'краткое изложение на русском языке, точно передавая '
                          'смысл без искажений и отклонений от оригинала.',
               'role': 'system'},
              {'content': 'Сократи текст.\n'
                          ' Картер Пейдж в нулевые годы занимался бизнесом в '
                          'России Отвечая на вопрос демократа из штата '
                          'Калифорния Адама Шиффа, он признал, что контакт '
                          'состоялся на конференции по случаю очередного '
                          'выпуска в Российской экономической школе в Сколкове '
                          'в июле 2016 года, сообщила New York Times. Пейдж '
                          'подтвердил данную информацию телеканалу Си-эн-эн '
                          'вечером в пятницу. По его словам, это был

In [8]:
pprint(test_jsonl[0])

{'messages': [{'content': 'Ты — ассистент, который помогает человеку делать '
                          'суммаризацию текста.Твоя задача — формулировать '
                          'краткое изложение на русском языке, точно передавая '
                          'смысл без искажений и отклонений от оригинала.',
               'role': 'system'},
              {'content': 'Кратко суммаризируй текст:\n'
                          '"Если девушка выбирает свой первый мотоцикл, нужно '
                          'учесть множество разных факторов. Необходимо '
                          'ориентироваться на то, является ли удобным '
                          'положение за рулём. Желательно взять ТС на '
                          'несколько часов и протестировать, чтобы убедиться в '
                          'отсутствии проблем. Хорошо, если можно опробовать '
                          'несколько разных мотоциклов и для себя понять, '
                          'какая между ними разница.\n'
       

In [9]:
def split_train_val(
    data: list,
    train_ratio: float = 0.8,
    seed: int = 42,
    train_size: int | None = None,
    val_size: int | None = None
) -> tuple[list, list]:
    dataset_size = len(data)

    rng = random.Random(seed)
    indices = list(range(dataset_size))
    rng.shuffle(indices)

    if train_size is not None and val_size is not None:
        subset_size = train_size + val_size
        assert 0 < subset_size <= dataset_size, f"train_size + val_size должно быть в диапазоне [1, {dataset_size}]"

        subset_indices = indices[:subset_size]
        train_indices = subset_indices[:train_size]
        val_indices = subset_indices[train_size:train_size + val_size]

    else:
        assert 0.0 < train_ratio < 1.0, "train_ratio должен быть между 0 и 1"

        split_idx = int(dataset_size * train_ratio)
        
        train_indices = indices[:split_idx]
        val_indices = indices[split_idx:]

    train_list = [data[i] for i in train_indices]
    val_list = [data[i] for i in val_indices]

    return train_list, val_list


In [10]:
train_size = 10_000
val_size = 2000

train_jsonl, val_jsonl = split_train_val(train_jsonl, train_size=train_size, val_size=val_size)

assert len(train_jsonl) == train_size
assert len(val_jsonl) == val_size

In [11]:
def write_to_jsonl_file(file_path: str, data: list[dict]):
    with open(file_path, "w") as file:
        for messages in tqdm(data):
            json_line = json.dumps(messages, ensure_ascii=False)
            file.write(json_line + "\n")

In [12]:
write_to_jsonl_file("../../data/train.jsonl", train_jsonl)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
write_to_jsonl_file("../../data/val.jsonl", val_jsonl)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [14]:
write_to_jsonl_file("../../data/test.jsonl", test_jsonl)

  0%|          | 0/258 [00:00<?, ?it/s]